In [1]:
!rm -rf /content/repo
!git clone https://github.com/malikee00/hm-recommender-system.git /content/repo

%cd "/content/repo/H&M"
!pwd
!ls -la

Cloning into '/content/repo'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 55 (delta 15), reused 46 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 24.73 KiB | 12.36 MiB/s, done.
Resolving deltas: 100% (15/15), done.
[Errno 2] No such file or directory: '/content/repo/H&M'
/content
/content
total 20
drwxr-xr-x 1 root root 4096 Jan 15 02:50 .
drwxr-xr-x 1 root root 4096 Jan 15 02:49 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 9 root root 4096 Jan 15 02:50 repo
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data


In [2]:
!pip -q install -U pip
!pip -q install numpy pandas pyarrow tqdm faiss-cpu gdown

import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00a 0:00:01
torch: 2.9.0+cu126
cuda available: True


In [13]:
import os

FEATURE_STORE_DIR = "data/feature_store"
os.makedirs(FEATURE_STORE_DIR, exist_ok=True)

ZIP_PATH = "/content/feature_store.zip"
FILE_ID = "1T-G1_Aa--Pd-xvQBBFO2bpt1XQm8PrRs"
!gdown --id $FILE_ID -O $ZIP_PATH
!ls -lh $ZIP_PATH

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1T-G1_Aa--Pd-xvQBBFO2bpt1XQm8PrRs
From (redirected): https://drive.google.com/uc?id=1T-G1_Aa--Pd-xvQBBFO2bpt1XQm8PrRs&confirm=t&uuid=27ce7673-82e8-46f6-8fdd-7b36ca5cd9e7
To: /content/feature_store.zip
100% 694M/694M [00:14<00:00, 47.4MB/s] 
-rw-r--r-- 1 root root 662M Jan 15 03:13 /content/feature_store.zip


In [14]:
!unzip -o /content/feature_store.zip -d data/feature_store
!ls -lh data/feature_store | head

Archive:  /content/feature_store.zip
   creating: data/feature_store/feature_store/
   creating: data/feature_store/feature_store/interactions.parquet/
  inflating: data/feature_store/feature_store/interactions.parquet/.part-00000-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: data/feature_store/feature_store/interactions.parquet/.part-00001-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: data/feature_store/feature_store/interactions.parquet/.part-00002-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: data/feature_store/feature_store/interactions.parquet/.part-00003-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: data/feature_store/feature_store/interactions.parquet/.part-00004-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: data/feature_store/feature_store/interactions.parquet/.part-00005-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.cr

In [15]:
import pandas as pd

itx = pd.read_parquet("data/feature_store/interactions.parquet")
print(itx.head())
print("rows:", len(itx))
print("cols:", itx.columns.tolist())

                                         customer_id  article_id       t_dat  \
0  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   706016003  2019-11-29   
1  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   706016001  2019-11-29   
2  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   682236013  2019-11-29   
3  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   706016016  2019-11-29   
4  aaa7a0483dd5b9e395d95324dcbfeb617af9800f39487d...   783335003  2019-11-29   

      price  sales_channel_id  
0  0.025288                 2  
1  0.025288                 2  
2  0.018983                 2  
3  0.025288                 2  
4  0.020322                 2  
rows: 31788324
cols: ['customer_id', 'article_id', 't_dat', 'price', 'sales_channel_id']


In [18]:
!python -m ml.recommender.train \
  --feature_store_dir data/feature_store \
  --registry_dir ml/registry/recommender \
  --embedding_dim 64 \
  --epochs 2 \
  --batch_size 2048 \
  --max_interactions 2000000

epoch 1/2:   0% 0/976 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/repo/ml/recommender/train.py", line 291, in <module>
    main()
  File "/content/repo/ml/recommender/train.py", line 203, in main
    u_feat = user_feat_mat_t[u_idx].to(device)
             ~~~~~~~~~~~~~~~^^^^^^^
RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)


In [ ]:
!python -m ml.recommender.evaluate \
  --feature_store_dir data/feature_store \
  --registry_dir ml/registry/recommender \
  --reports_dir ml/reports/recommender \
  --ks 5,10,20

In [ ]:
!rm -f recommender_artifacts.zip
!zip -r recommender_artifacts.zip ml/registry/recommender ml/reports/recommender
!ls -lh recommender_artifacts.zip

from google.colab import files
files.download("recommender_artifacts.zip")

In [12]:
!unzip -o /content/feature_store.zip -d data/feature_store

Archive:  /content/feature_store.zip
   creating: data/feature_store/item_popularity.parquet/
  inflating: data/feature_store/item_popularity.parquet/.part-00000-527b7ce3-00ab-4c30-91ab-86306f93169a-c000.snappy.parquet.crc  
  inflating: data/feature_store/item_popularity.parquet/._SUCCESS.crc  
  inflating: data/feature_store/item_popularity.parquet/part-00000-527b7ce3-00ab-4c30-91ab-86306f93169a-c000.snappy.parquet  
  inflating: data/feature_store/item_popularity.parquet/_SUCCESS  
   creating: data/feature_store/user_features.parquet/
  inflating: data/feature_store/user_features.parquet/.part-00000-95e03489-490d-4f0d-bae1-eaa4214d2e0b-c000.snappy.parquet.crc  
  inflating: data/feature_store/user_features.parquet/.part-00001-95e03489-490d-4f0d-bae1-eaa4214d2e0b-c000.snappy.parquet.crc  
  inflating: data/feature_store/user_features.parquet/.part-00002-95e03489-490d-4f0d-bae1-eaa4214d2e0b-c000.snappy.parquet.crc  
  inflating: data/feature_store/user_features.parquet/.part-00003-95

In [ ]:
!ls -lh data/feature_store

In [9]:
!python -m ml.recommender.train \
  --feature_store_dir data/feature_store \
  --registry_dir ml/registry/recommender \
  --embedding_dim 64 \
  --epochs 2 \
  --batch_size 2048 \
  --max_interactions 2000000

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/repo/ml/recommender/train.py", line 291, in <module>
    main()
  File "/content/repo/ml/recommender/train.py", line 140, in main
    interactions, user_df, item_df, item_popularity = load_training_tables(args.feature_store_dir)
                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/repo/ml/recommender/train.py", line 32, in load_training_tables
    interactions = pd.read_parquet(os.path.join(feature_store_dir, "interactions.parquet"))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 667, in read_parquet
    return impl.read(
           ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 267, in read
    path_or_handle, 

In [ ]:
!python -m ml.recommender.evaluate \
  --feature_store_dir data/feature_store \
  --registry_dir ml/registry/recommender \
  --reports_dir ml/reports/recommender \
  --ks 5,10,20


In [ ]:
!rm -f recommender_artifacts.zip
!zip -r recommender_artifacts.zip ml/registry/recommender ml/reports/recommender
!ls -lh recommender_artifacts.zip

from google.colab import files
files.download("recommender_artifacts.zip")
